In [ ]:
!pip install scipy pandas numpy matplotlib scikit-learn fsspec huggingface_hub

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as lm

# Load the data
df = pd.read_csv("hf://datasets/hugginglearners/data-science-job-salaries/ds_salaries.csv")

In [ ]:
# 1. GENERAL SALARY DATA:

In [ ]:
df['salary_in_usd'].describe()

In [ ]:
plt.boxplot(df['salary_in_usd'], vert=False)
plt.title('Box Plot of Salary in USD')
plt.xlabel('Salary')
plt.show()

In [ ]:
plt.hist(df['salary_in_usd'])
plt.title('Historgram of Salary in USD')
plt.xlabel('Salary')
plt.ylabel('frequency')
plt.show()

As seen in the Box Plot and Historam above, the mean is greater than the median. The visualizations show a tail on the left side meaning a right-skewed distribution and there is a single peak in the second bin of the histogram. It is easy in the box plot to see outliers to the left (outliers of very high salaries). The salaries in usd range from 2859 to 600000 with the the middle 50% of the salaries falling between 62726 (Q1) and 150000 (Q3).

In [ ]:
# 2 Effect of Experience on Salary

In [ ]:
df.boxplot(column='salary_in_usd', by='experience_level', vert=False, grid=False)
plt.title('Side-by-side box plot of salary by experience level')
plt.xlabel('salary in usd')
plt.ylabel('experience level')
plt.show()

In [ ]:
mean_salary_by_experience = df.groupby('experience_level')['salary_in_usd'].describe()
print(mean_salary_by_experience)

All experience levels have a few outliers of higher salary values. From the data, we see that EX level has the highest mean and median. The salaries for the EX level tend to be higher over all. The box plots show that even Q1 for the EX level is greater than Q3 for experience levels EN and MI. Comparing mean salary values from highest to lowest it goes: EX, SE, MI, and then EN. The salaries for those with EN experience tend to be the lowest as seen in the box plot by looking at the IQR being the smallest and having the lowest values. The data overall seems to be consistent for all experience levels that they are slightly skewed to the right, some experience levels more skewed than others. The mean for each experience level is higher than the median. Although still skewed to the right, the salaries for experience level of SE is most symmetric. From this data, we can conclude that their is a relationship between salary and experience level.

In [ ]:
# 3 Impact of Company Size on Salary

sizes = ['S', 'M', 'L']
salary_data = [df[df['company_size'] == size]['salary_in_usd'] for size in sizes]

fig, ax = plt.subplots(figsize=(8, 4))

bp = ax.boxplot(salary_data, labels=sizes, patch_artist=True)

colors = ['lightblue', 'lightgreen', 'lightpink']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

ax.set_title('Salary by Company Size', fontsize=16)
ax.set_xlabel('Company Size', fontsize=12)
ax.set_ylabel('Salary (USD)', fontsize=12)

ax.yaxis.grid(True, linestyle='--', color='grey')

plt.show()

In [ ]:
# 4 Job Title Analysis
job_df = df[['job_title', 'salary_in_usd']]


jobs = {}

jobs['Data Scientist Roles'] = ['Data Scientist', 'Principal Data Scientist', 'Staff Data Scientist', 'Applied Data Scientist', 'Data Science Engineer', 'Lead Data Scientist', 'Data Science Consultant']
jobs['Data Engineer Roles'] = ['Data Engineer', 'Lead Data Engineer', 'Principal Data Engineer', 'Big Data Engineer', 'Cloud Data Engineer', 'Data Engineering Manager', 'ETL Developer', 'Data Architect', 'Big Data Architect']
jobs['Machine Learning Roles'] = ['Machine Learning Engineer', 'Applied Machine Learning Scientist', 'ML Engineer', 'Machine Learning Scientist', 'Machine Learning Developer', 'Machine Learning Infrastructure Engineer', 'Lead Machine Learning Engineer', 'NLP Engineer', 'Machine Learning Manager', 'Head of Machine Learning']
jobs['Analyst Roles'] = ['Data Analyst','Principal Data Analyst','Finance Data Analyst', 'Business Data Analyst', 'BI Data Analyst', 'Lead Data Analyst', 'Financial Data Analyst', 'Product Data Analyst', 'Marketing Data Analyst']
jobs['Management Roles'] = ['Data Science Manager', 'Data Analytics Manager', 'Director of Data Science', 'Head of Data', 'Head of Data Science', 'Director of Data Engineering', 'Data Engineering Manager', 'Machine Learning Manager', 'Data Analytics Lead']
jobs['Research Advanced Roles'] = ['Research Scientist', 'AI Scientist', '3D Computer Vision Researcher', 'Computer Vision Engineer', 'Computer Vision Software Engineer']
jobs['Specialized Roles'] = ['Data Specialist', 'Data Analytics Engineer', 'Analytics Engineer', 'Financial Data Analyst']



# replace job titles with categories
flag = False
job_title = "none"
for row in job_df.itertuples():
    for key, value in jobs.items():
        job_title = row.job_title
        if row.job_title in value:
            job_df.at[row.Index, 'job_title'] = key
            break


# Group salaries by job titles
job_titles = job_df['job_title'].unique()
job_df = [job_df[job_df['job_title'] == title]['salary_in_usd'] for title in job_titles]
plt.violinplot(job_df)
plt.xticks(ticks=range(1, len(job_titles) + 1), labels=job_titles, rotation=90)
plt.title('Mass Distrobution of Salaries by Job Type')
plt.show()

In [ ]:
# 5 Interactions Between Relationships
df_5 = df
df_5['salary_in_usd'] = pd.to_numeric(df_5['salary_in_usd'], errors='coerce')
df_5 = df_5.dropna(subset=['job_title', 'company_size', 'salary_in_usd'])

pivot_table = pd.pivot_table(df_5, 
                             values='salary_in_usd', 
                             index='job_title', 
                             columns='company_size', 
                             aggfunc='mean')

pivot_table = pivot_table.round(-3)
pivot_table['mean'] = pivot_table.mean(axis=1)
pivot_table = pivot_table.sort_values('mean', ascending=False)
pivot_table = pivot_table.drop('mean', axis=1)
print(pivot_table.head(23).to_string())

summary_stats = df.groupby('company_size')['salary_in_usd'].agg(['mean', 'median', 'std', 'count'])
summary_stats = summary_stats.round(0)

print("\nSummary Statistics by Company Size:")
print(summary_stats.to_string())

exp_summary = df.groupby(['experience_level', 'company_size'])['salary_in_usd'].mean().unstack()
exp_summary = exp_summary.round(-3)

print("\nMean Salary by Experience Level and Company Size:")
print(exp_summary.to_string())